In [3]:
%cd ~/dissertation_code 
import sys
sys.path.append("RQ 3/")
import pandas as pd
import numpy as np
from scipy import stats
import itertools

/home/tntech.edu/kebrown46/dissertation_code


In [6]:
def welch_ttest(x, y, alternative): 
    ## Welch-Satterthwaite Degrees of Freedom ##
    dof = (x.var()/x.size + y.var()/y.size)**2 / ((x.var()/x.size)**2 / (x.size-1) + (y.var()/y.size)**2 / (y.size-1))
   
    t, p = stats.ttest_ind(x, y, alternative=alternative, equal_var = False)
    
    print("\n",
          f"Welch's t-test= {t:.4f}", "\n",
          f"p-value = {p}", "\n",
          f"Welch-Satterthwaite Degrees of Freedom= {dof:.4f}")
    return t, p, dof

def evaluate(model, dataset, mode):
    print("====={0} - {1} - {2}=====".format(model, dataset, mode))
    dfs = []
    for i in range(10):
        fn = "RQ 3/results/subgroup_perf_{0}_{1}_{2}_{3}.csv".format(model, dataset, i, mode)
        df = pd.read_csv(fn)
        dfs.append(df[["subgroup", "subgroup size", "accuracy diff", "primary bin predicted"]])

    df = pd.concat(dfs, axis=0)

    zeros_subgroups = df[df["primary bin predicted"] == 0]
    ones_subgroups = df[df["primary bin predicted"] == 1]
    X0 = zeros_subgroups["accuracy diff"].values
    X1 = ones_subgroups["accuracy diff"].values
    one_weights = ones_subgroups["subgroup size"].values
    zero_weights = zeros_subgroups["subgroup size"].values
    print("Mean(X0) = {0}".format(np.average(X0, weights=zero_weights)))
    print("Mean(X1) = {0}".format(np.average(X1, weights=one_weights)))

    return welch_ttest(X1, X0, "greater")

In [7]:
# model = "NN-dropout"
# dataset = "trauma_uk"
mode = "remove"
models = ["NN-dropout", "catboost-ve"]
datasets = ["trauma_uk", "diabetes"]
combos = list(itertools.product(models, datasets))

for combo in combos:
    model, dataset = combo
    evaluate(model, dataset, mode)

=====NN-dropout - trauma_uk - remove=====
Mean(X0) = -0.03596926182853768
Mean(X1) = 0.01713685495767408

 Welch's t-test= 4.8948 
 p-value = 1.3408990320615499e-06 
 Welch-Satterthwaite Degrees of Freedom= 139.5361
=====NN-dropout - diabetes - remove=====
Mean(X0) = -0.01586109579472458
Mean(X1) = 0.013868823023470773

 Welch's t-test= 1.7234 
 p-value = 0.046331834810321076 
 Welch-Satterthwaite Degrees of Freedom= 39.4130
=====catboost-ve - trauma_uk - remove=====
Mean(X0) = -0.006902130302096575
Mean(X1) = 0.004631150036336837

 Welch's t-test= 5.6475 
 p-value = 1.568500939682211e-08 
 Welch-Satterthwaite Degrees of Freedom= 391.1865
=====catboost-ve - diabetes - remove=====


FileNotFoundError: [Errno 2] No such file or directory: 'RQ 3/results/subgroup_perf_catboost-ve_diabetes_0_remove.csv'

In [46]:
model = "catboost-ve"
dataset = "trauma_uk"
mode = "remove"



Mean(X0) = -0.0019399431714332625
Mean(X1) = 0.0010893028584513644

 Welch's t-test= 5.6475 
 p-value = 1.568500939682211e-08 
 Welch-Satterthwaite Degrees of Freedom= 391.1865


(5.6475342594457745, 1.568500939682211e-08, 391.18647050126435)